In [ ]:
## setup analysis
execfile(r'D:\measuring\analysis\scripts\setup_analysis.py')
from analysis.lib.purification import purify_pq as ppq; reload(ppq)
from analysis.lib.purification import purify_analysis as pa; reload(pa)
%matplotlib inline

theta = 'pi/8'


In [ ]:
folder_lt3,folder_lt4 = pa.get_folders_from_theta(theta)
ssro_calib_lt3,ssro_calib_lt4 = pa.get_ssro_calibs()

Purify_XX = pa.purify_analysis('purification_analysis',folder_lt3,folder_lt4,ssro_calib_lt3,ssro_calib_lt4)
Purify_YY = pa.purify_analysis('purification_analysis',folder_lt3,folder_lt4,ssro_calib_lt3,ssro_calib_lt4)
Purify_ZZ = pa.purify_analysis('purification_analysis',folder_lt3,folder_lt4,ssro_calib_lt3,ssro_calib_lt4)

Purify_XX.get_tstamps_and_offsets(contains = 'XX')
Purify_YY.get_tstamps_and_offsets(contains = 'YY')
Purify_ZZ.get_tstamps_and_offsets(contains = 'ZZ')


In [ ]:
### load the data
Purify_XX.load_raw_data()
Purify_XX.correct_pq_times()
Purify_YY.load_raw_data()
Purify_YY.correct_pq_times()
Purify_ZZ.load_raw_data()
Purify_ZZ.correct_pq_times()

In [ ]:
def sweep_parameter(param_name,sweep_pts,apply_ROC, plot_all = False):
    
    ## get correlation probabilties
    psi_m_XX,psi_p_XX,psi_m_XX_corrs,psi_p_XX_corrs = Purify_XX.sweep_filter_parameter_vs_correlations(param_name,sweep_pts,
                                                                         apply_ROC = apply_ROC,do_plot= False)
    psi_m_YY,psi_p_YY,psi_m_YY_corrs,psi_p_YY_corrs = Purify_YY.sweep_filter_parameter_vs_correlations(param_name,sweep_pts,
                                                                         apply_ROC = apply_ROC,do_plot= False)
    psi_m_ZZ,psi_p_ZZ,psi_m_ZZ_corrs,psi_p_ZZ_corrs = Purify_ZZ.sweep_filter_parameter_vs_correlations(param_name,sweep_pts,
                                                                         apply_ROC = apply_ROC,do_plot= False)       
    ## calculate fidelities
    psi_m_F = (psi_m_XX[1] + psi_m_YY[1] + psi_m_ZZ[1]+1)/4.
    psi_p_F = (psi_p_XX[1] + psi_p_YY[1] + psi_p_ZZ[1]+1)/4.
      
    ## calciulate error bars
    psi_m_F_u = np.sqrt(psi_m_XX[2]**2 + psi_m_YY[2]**2 + psi_m_ZZ[2]**2)/4.
    psi_p_F_u = np.sqrt(psi_p_XX[2]**2 + psi_p_YY[2]**2 + psi_p_ZZ[2]**2)/4.
    
    # calculate ebits    
    ebits_p, ebits_p_u = pa.calculate_ebits(psi_m_YY,psi_m_ZZ,psi_m_XX_corrs)
    ebits_m, ebits_m_u = pa.calculate_ebits(psi_p_YY,psi_p_ZZ,psi_p_XX_corrs)
    
    psi_F = 0.5*(psi_m_F+psi_p_F)
    psi_F_u = 0.5*np.sqrt(psi_m_F_u**2+psi_p_F_u**2)
    
    
    fig  = plt.figure()
    ax = plt.subplot()
    ax.set_xlabel(param_name)
    ax.set_ylabel('Fidelity')
    plt.errorbar(sweep_pts,psi_m_F,psi_m_F_u,label = '-')
    plt.errorbar(sweep_pts,psi_p_F,psi_p_F_u,label = '+')
    if param_name == 'bin_w2':
        plt.xlim(np.array(plt.xlim()) + [-50,50])
    plt.legend()
    plt.show()
    plt.close('all')
 
    fig  = plt.figure()
    ax = plt.subplot()
    ax.set_xlabel(param_name)
    ax.set_ylabel('Fidelity (avg)')
    plt.errorbar(sweep_pts,psi_F,psi_F_u)
    if param_name == 'bin_w2':
        plt.xlim(np.array(plt.xlim()) + [-50,50])
    plt.show()
    plt.close('all')
    
    fig  = plt.figure()
    ax = plt.subplot()
    ax.set_xlabel(param_name)
    ax.set_ylabel('ebits')

    plt.errorbar(sweep_pts,ebits_p,ebits_p_u,label = '+')
    plt.errorbar(sweep_pts,ebits_m,ebits_m_u,label = '-')
    if param_name == 'bin_w2':
        plt.xlim(np.array(plt.xlim()) + [-50,50])
    plt.legend()
    plt.show()
    plt.close('all')
    
    ### get the rate for any given data point
    ### keep in mind that this includes ALL down time of the experiment. Some might argue that this is an unfair comparison
    ### we therefore also include a diagram that uses the operation time of the sequence only and the number of times the sequence has run
    ### tomography is excluded from this calculated rate
    total_time = 0
    total_time += Purify_XX.get_total_time() + Purify_YY.get_total_time() + Purify_ZZ.get_total_time()
    total_counts = psi_m_XX[0]+psi_p_XX[0]+psi_m_YY[0]+psi_p_YY[0]+psi_m_ZZ[0]+psi_p_ZZ[0]
    fig  = plt.figure()
    ax = plt.subplot()
    ax.set_xlabel(param_name)
    ax.set_ylabel('Rate (Hz)')
    plt.plot(sweep_pts,total_counts/total_time, label = 'incl. overhead')
    if param_name == 'bin_w2':
        plt.xlim(np.array(plt.xlim()) + [-50,50]) 
    total_time = 0
    total_time += Purify_XX.estimate_sequence_time() + Purify_YY.estimate_sequence_time() + Purify_ZZ.estimate_sequence_time()
    
    plt.plot(sweep_pts,total_counts/total_time, label = 'excl. overhead')
    if param_name == 'bin_w2':
        plt.xlim(np.array(plt.xlim()) + [-50,50])
    plt.legend(loc = 2)
    plt.show()
    plt.close('all')
    
    ## plot individual correlations
    if plot_all:
        fig  = plt.figure()
        ax = plt.subplot()
        ax.set_xlabel(param_name)
        ax.set_ylabel('Expectation value')
        plt.errorbar(sweep_pts,psi_m_XX[1],psi_m_XX[2],label = 'XX')
        plt.errorbar(sweep_pts,psi_m_YY[1],psi_m_YY[2],label = 'YY')
        plt.errorbar(sweep_pts,psi_m_ZZ[1],psi_m_ZZ[2],label = 'ZZ')
        if param_name == 'bin_w2':
            plt.xlim(np.array(plt.xlim()) + [-50,50])   
        plt.title('Psi_minus correlations')
        plt.legend()
        plt.show()
        plt.close('all')
        
        fig  = plt.figure()
        ax = plt.subplot()
        ax.set_xlabel(param_name)
        ax.set_ylabel('Expectation value')
        plt.errorbar(sweep_pts,psi_p_XX[1],psi_p_XX[2],label = 'XX')
        plt.errorbar(sweep_pts,psi_p_YY[1],psi_p_YY[2],label = 'YY')
        plt.errorbar(sweep_pts,psi_p_ZZ[1],psi_p_ZZ[2],label = 'ZZ')
        if param_name == 'bin_w2':
            plt.xlim(np.array(plt.xlim()) + [-50,50])
        plt.title('Psi_plus correlations')
        plt.legend()
        plt.show()
        plt.close('all')

In [ ]:
sweep_parameter('bin_w2',np.arange(50,450,50),apply_ROC = True,plot_all = True)

In [ ]:
sweep_parameter('st_start',np.linspace(-4e3,4e3,10)+2773.5e3,apply_ROC = True)

In [ ]:
#### some plotting globals
lw = 1.
fontsize = 11.
markersize = 3
color_list = ['#4FA6FF','#50A53A','orange','Gray','yellow']

#### compare correlations in one plot + nicer plotting
#### also gets the fidelity etc.
#### we only look at average correlations (so averaged over the plus and minus signature).
def load_data(theta):
    folder_lt3,folder_lt4 = pa.get_folders_from_theta(theta)
    ssro_calib_lt3,ssro_calib_lt4 = pa.get_ssro_calibs()

    Purify_XX = pa.purify_analysis('purification_analysis',folder_lt3,folder_lt4,ssro_calib_lt3,ssro_calib_lt4)
    Purify_YY = pa.purify_analysis('purification_analysis',folder_lt3,folder_lt4,ssro_calib_lt3,ssro_calib_lt4)
    Purify_ZZ = pa.purify_analysis('purification_analysis',folder_lt3,folder_lt4,ssro_calib_lt3,ssro_calib_lt4)

    Purify_XX.get_tstamps_and_offsets(contains = 'XX')
    Purify_YY.get_tstamps_and_offsets(contains = 'YY')
    Purify_ZZ.get_tstamps_and_offsets(contains = 'ZZ')
    
    Purify_XX.load_raw_data()
    Purify_XX.correct_pq_times()
    Purify_YY.load_raw_data()
    Purify_YY.correct_pq_times()
    Purify_ZZ.load_raw_data()
    Purify_ZZ.correct_pq_times()
    
    return Purify_XX,Purify_YY,Purify_ZZ
    
def get_corrs(param_name,sweep_pts,Purify_XX,Purify_YY,Purify_ZZ,apply_ROC = True):
    ## get correlation probabilties
    psi_m_XX,psi_p_XX,psi_m_XX_corrs,psi_p_XX_corrs = Purify_XX.sweep_filter_parameter_vs_correlations(param_name,sweep_pts,
                                                                         apply_ROC = apply_ROC,do_plot= False)
    psi_m_YY,psi_p_YY,psi_m_YY_corrs,psi_p_YY_corrs = Purify_YY.sweep_filter_parameter_vs_correlations(param_name,sweep_pts,
                                                                         apply_ROC = apply_ROC,do_plot= False)
    psi_m_ZZ,psi_p_ZZ,psi_m_ZZ_corrs,psi_p_ZZ_corrs = Purify_ZZ.sweep_filter_parameter_vs_correlations(param_name,sweep_pts,
                                                                         apply_ROC = apply_ROC,do_plot= False)       
    ## calculate fidelities
    psi_m_F = (psi_m_XX[1] + psi_m_YY[1] + psi_m_ZZ[1]+1)/4.
    psi_p_F = (psi_p_XX[1] + psi_p_YY[1] + psi_p_ZZ[1]+1)/4.
      
    ## calciulate error bars
    psi_m_F_u = np.sqrt(psi_m_XX[2]**2 + psi_m_YY[2]**2 + psi_m_ZZ[2]**2)/4.
    psi_p_F_u = np.sqrt(psi_p_XX[2]**2 + psi_p_YY[2]**2 + psi_p_ZZ[2]**2)/4.
    
    # calculate ebits    
    ebits_p, ebits_p_u = pa.calculate_ebits(psi_m_YY,psi_m_ZZ,psi_m_XX_corrs)
    ebits_m, ebits_m_u = pa.calculate_ebits(psi_p_YY,psi_p_ZZ,psi_p_XX_corrs)
    
    psi_F = 0.5*(psi_m_F+psi_p_F)
    psi_F_u = 0.5*np.sqrt(psi_m_F_u**2+psi_p_F_u**2)
    
    psi_XX    = (psi_m_XX[1]+psi_p_XX[1])/2.
    psi_YY    = (psi_m_YY[1]+psi_p_YY[1])/2.
    psi_ZZ    = (psi_m_ZZ[1]+psi_p_ZZ[1])/2.
    psi_XX_u  = np.sqrt(psi_m_XX[2]**2+psi_p_XX[2]**2)/2.
    psi_YY_u  = np.sqrt(psi_m_YY[2]**2+psi_p_YY[2]**2)/2.
    psi_ZZ_u  = np.sqrt(psi_m_ZZ[2]**2+psi_p_ZZ[2]**2)/2.
    
    return psi_F,psi_F_u,psi_XX,psi_XX_u,psi_YY,psi_YY_u,psi_ZZ,psi_ZZ_u

def plot_values_bars(x_list,y_list,y_u_list):
    
    for x,y,y_u,color in zip(x_list,y_list,y_u_list,color_list):
        plt.errorbar(x,y,y_u,fmt = 'o',color = color,ms=markersize,mew=lw,zorder=3) #formatting still needs to be done
        ### we want to include bars to signify integration here
        barwidth = x[1]-x[0]
        plt.bar(x-barwidth,y,width = barwidth,alpha = 0.1,color = color,zorder=0)
        
def plot_model_fidelity(theta):
    
    if theta == 'pi/8':
        model_x =  np.arange(50,450,50)
        model_y = np.array([0.69838,0.628115,0.550945,0.4897,0.451624,0.430624,0.418807,0.41139])
    elif theta == 'pi/6':
        model_x =  np.arange(50,450,50)
        model_y = np.array([0.671159,0.588074, 0.502691,0.440062, 0.40325, 0.38368, 0.373077, 0.366722])
    elif theta == 'pi/5':
        model_x =  np.arange(50,450,50)
        model_y = np.array([0.640959, 0.546748, 0.456214, 0.393848, 0.358362, 0.339751, 0.3299, 0.32427])
    elif theta == 'pi/4':
        model_x =  np.arange(50,450,50)
        model_y = np.array([0.580464, 0.472287, 0.379295,0.320023, 0.286901, 0.269389, 0.260286,0.255461])
        
    plt.plot(model_x,model_y,'--',color=color_list[0],lw=lw,zorder=1)

In [ ]:
####
theta = 'pi/6' #'pi/8'
XX,YY,ZZ = load_data(theta)

In [ ]:
x_sweep = np.arange(50,450,50)
psi_F,psi_F_u,psi_XX,psi_XX_u,psi_YY,psi_YY_u,psi_ZZ,psi_ZZ_u = get_corrs('bin_w2',x_sweep,XX,YY,ZZ,apply_ROC = True)

In [ ]:
y_list = [psi_F,psi_XX,psi_YY,psi_ZZ];
x_list = [x_sweep]*len(y_list);
y_u_list = [psi_F_u,psi_XX_u,psi_YY_u,psi_ZZ_u];

In [ ]:
import matplotlib.lines as mlines

save_folder = r'K:\ns\qt\Diamond\Projects\Purification\Paper\Plots'
golden_ratio = 1.61803
mm_to_inches = 0.0393701 ### inch/mm
figure_width = mm_to_inches*80
fig = plt.figure(figsize =(figure_width,figure_width/golden_ratio))
ax = plt.subplot(111)

color_list = ['#4FA6FF','#50A53A','orange','Gray']

plot_values_bars(x_list,y_list,y_u_list)
plot_model_fidelity(theta)

ax.tick_params(labelsize = fontsize,width =lw)
# ax.set_ylabel('Expectation value',size = fontsize)
ax.set_xlabel('N (binned)',size = fontsize)
# ax.set_ylabel('Expectation value / Fidelity')
ax.set_xticks([0,100,200,300,400])
ax.set_ylim([0,1])
ax.set_xlim([0,420])

### formatting of the legend
legend_list = ['Fidelity',r'$\langle \hat{X}\hat{X} \rangle$',r'$\langle \hat{Y}\hat{Y} \rangle$',r'$\langle \hat{Z}\hat{Z} \rangle$']
Fid = mlines.Line2D([], [], color=color_list[0], marker='o',
                          markersize=markersize,mew=lw, label=legend_list[0])
XX = mlines.Line2D([], [], color=color_list[1], marker='o',mew=lw,
                          markersize=markersize, label=legend_list[1])
YY = mlines.Line2D([], [], color=color_list[2], marker='o',mew=lw,
                          markersize=markersize, label=legend_list[2])
ZZ = mlines.Line2D([], [], color=color_list[3], marker='o',mew=lw,
                          markersize=markersize, label=legend_list[3])
# ax.legend(legend_list, loc=2, borderaxespad=0.,fontsize =fontsize )
ax.legend(handles=[Fid,XX,YY,ZZ],ncol=2,numpoints = 1,frameon=False,
                                  handlelength=0,handletextpad = 0.4,columnspacing = 0.8,
                                  borderaxespad =0.05,
                                  fontsize=fontsize-1)

ax.xaxis.set_tick_params(width=lw)
ax.yaxis.set_tick_params(width=lw)

[i.set_linewidth(lw) for i in ax.spines.itervalues()]
plt.savefig(os.path.join(save_folder,'binned_correlations_'+theta[-1]+'.png'),format='png',bbox_inches = 'tight',pad_inches=0.3)
plt.savefig(os.path.join(save_folder,'binned_correlations_'+theta[-1]+'.pdf'),format='pdf',bbox_inches = 'tight',pad_inches=0.3)
plt.show()


In [ ]:
ax.legend?